In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select


In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--headless')  #ẩn
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-extensions");
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

In [4]:
url='https://platform.who.int/mortality/themes/theme-details/topics/topic-details/MDB/infectious-and-parasitic-diseases'
#sẽ sửa thành 1 list các nguyên nhân
# Infectious and parasitic diseases

In [5]:
driver.get(url)

In [6]:
#chọn dạng biểu đồ
driver.find_element(By.XPATH, '//*[@id="visualization_component_4_key"]').click()

In [7]:
#chọn giới tính
driver.implicitly_wait(3)
select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[5]')
select = Select(select.find_element(By.CLASS_NAME,'form-control'))

In [8]:
#Chọn gt làm nam
select.select_by_visible_text('Male')

In [9]:
#chọn nhóm tuổi
driver.implicitly_wait(10)
select=driver.find_element(By.XPATH, '//*[@id="visualization_component_htmlid"]/div/div[2]/div[1]/div[4]')
select = Select(select.find_element(By.CLASS_NAME,'form-control'))
select.select_by_visible_text('0')

In [10]:
#chọn quốc gia
driver.implicitly_wait(30)
driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key"]/input').click()
driver.implicitly_wait(30)
driver.find_element(By.XPATH, '//*[@id="filter_indicatorlinechart_countries7_key_ul_level1"]/li[1]/label/input').click()

In [11]:
#Filter
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="filters_section_indicatorlinechart_countries_key_filter_button"]').click()
#đợi load
time.sleep(2)

In [12]:
table_id = driver.find_element(By.ID, 'custom_table')
cols = table_id.find_elements(By.TAG_NAME, "th")


In [13]:
#lấy số quốc gia
country=[]
for i in cols:
    country.append(i.text)
country=country[1:]
country

['Albania',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Cabo Verde',
 'Canada',
 'Chile',
 'China, Hong Kong SAR',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'Georgia',
 'Germany',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guatemala',
 'Guyana',
 'Hungary',
 'Iceland',
 'Iran (Islamic Republic of)',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lebanon',
 'Lithuania',
 'Luxembourg',
 'Maldives',
 'Malta',
 'Martinique',
 'Mauritius',
 'Mayotte',
 'Mexico',
 'Mongolia',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Nicaragua',
 'North Mace

In [14]:
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]
len(rows)

10

In [15]:
#lấy năm
#year=[]
#for i in range (1,11):
#    year.append(driver.find_element(By.XPATH,'//*[@id="custom_table_body"]/tr['+str(i)+']/td[1]').text)
#year

['2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011']

In [16]:
driver.find_element(By.XPATH, '//*[@id="custom_table_body"]/tr[1]/td[2]').text

''

In [17]:
df=pd.DataFrame()
for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])
        
    
    

{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Albania', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Antigua and Barbuda', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Argentina', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Armenia', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic

{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Greece', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Grenada', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guadeloupe', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guatemala', 'Number of deaths': 288}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases'

{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Slovenia', 'Number of deaths': 0}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'South Africa', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Spain', 'Number of deaths': 8}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Sri Lanka', 'Number of deaths': nan}
{'Year': '2020', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 

{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Denmark', 'Number of deaths': nan}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Dominica', 'Number of deaths': nan}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Dominican Republic', 'Number of deaths': nan}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Ecuador', 'Number of deaths': 38}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic di

{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Netherlands', 'Number of deaths': 8}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'New Zealand', 'Number of deaths': nan}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Nicaragua', 'Number of deaths': 59}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'North Macedonia', 'Number of deaths': 4}
{'Year': '2019', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic 

...

{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Egypt', 'Number of deaths': 2528}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'El Salvador', 'Number of deaths': 51}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Estonia', 'Number of deaths': 0}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Fiji', 'Number of deaths': 18}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Count

{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Norway', 'Number of deaths': 0}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Panama', 'Number of deaths': 40}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Paraguay', 'Number of deaths': 43}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Peru', 'Number of deaths': 202}
{'Year': '2012', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country':

{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Armenia', 'Number of deaths': 33}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Australia', 'Number of deaths': 9}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Austria', 'Number of deaths': 2}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Azerbaijan', 'Number of deaths': nan}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'C

{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Iceland', 'Number of deaths': 0}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Iran (Islamic Republic of)', 'Number of deaths': nan}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Iraq', 'Number of deaths': nan}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Ireland', 'Number of deaths': 5}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic d

{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Slovakia', 'Number of deaths': nan}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Slovenia', 'Number of deaths': 0}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'South Africa', 'Number of deaths': 3146}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Spain', 'Number of deaths': 14}
{'Year': '2011', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases',

In [18]:
df=df.reset_index()
display(df)

,index,Year,Sex,Cause Type,Cause Death,Country,Number of deaths
0,0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Albania,NaN
1,0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Antigua and Barbuda,NaN
2,0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Argentina,NaN
3,0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Armenia,NaN
4,0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Australia,1
...,...,...,...,...,...,...,...
1125,0,2011,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,United Kingdom,55
1126,0,2011,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,United States of America,349
1127,0,2011,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Uruguay,NaN
1128,0,2011,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Uzbekistan,185


In [21]:
driver.find_element(By.XPATH, '//*[@id="next_button"]').click()
time.sleep(0.5)
table_id = driver.find_element(By.ID, 'custom_table')

In [22]:
#lặp lại các bước
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]

for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        
        # cần tạo 1 cái gì đó để tra xem quốc gia đó thuộc khu vực nào và tạo trường area
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])

{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Albania', 'Number of deaths': 2}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Antigua and Barbuda', 'Number of deaths': nan}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Argentina', 'Number of deaths': 202}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Armenia', 'Number of deaths': 32}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic di

{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Greece', 'Number of deaths': 5}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Grenada', 'Number of deaths': 0}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guadeloupe', 'Number of deaths': 0}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guatemala', 'Number of deaths': 476}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Cou

{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Russian Federation', 'Number of deaths': 353}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Réunion', 'Number of deaths': 1}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Saint Kitts and Nevis', 'Number of deaths': 0}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Saint Lucia', 'Number of deaths': 1}
{'Year': '2010', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and 

{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Chile', 'Number of deaths': 28}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'China, Hong Kong SAR', 'Number of deaths': 6}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Colombia', 'Number of deaths': 273}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Costa Rica', 'Number of deaths': 8}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic dis

{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mauritius', 'Number of deaths': 3}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mayotte', 'Number of deaths': nan}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mexico', 'Number of deaths': 817}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mongolia', 'Number of deaths': nan}
{'Year': '2009', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'C

...

{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Japan', 'Number of deaths': 64}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Jordan', 'Number of deaths': nan}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Kazakhstan', 'Number of deaths': 163}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Kuwait', 'Number of deaths': 5}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Count

{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Spain', 'Number of deaths': 13}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Sri Lanka', 'Number of deaths': 132}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Suriname', 'Number of deaths': 8}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Sweden', 'Number of deaths': 5}
{'Year': '2002', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Countr

{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Costa Rica', 'Number of deaths': 24}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Croatia', 'Number of deaths': 2}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Cuba', 'Number of deaths': 31}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Cyprus', 'Number of deaths': nan}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Countr

{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mauritius', 'Number of deaths': 3}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mayotte', 'Number of deaths': nan}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mexico', 'Number of deaths': 1473}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Mongolia', 'Number of deaths': nan}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', '

{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Ukraine', 'Number of deaths': 132}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'United Kingdom', 'Number of deaths': 76}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'United States of America', 'Number of deaths': 348}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Uruguay', 'Number of deaths': 38}
{'Year': '2001', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and p

In [24]:
driver.find_element(By.XPATH, '//*[@id="next_button"]').click()
time.sleep(0.5)
table_id = driver.find_element(By.ID, 'custom_table')

In [25]:
#lặp lại các bước
#số dòng - bỏ header, header đã lưu vào biến col
rows=table_id.find_elements(By.TAG_NAME, "tr")[1:]

for r in rows:
    v={}
    values=r.find_elements(By.TAG_NAME, "td")
    v['Year']=values[0].text
    v['Sex']='Male'
    v['Cause Type']='Communicable, maternal, perinatal and nutritional conditions'
    v['Cause Death']='Infectious and parasitic diseases'
    for i in range (1,len(values)):
        v['Country']=country[i-1]
        if values[i].text=='':
            v['Number of deaths']=np.nan
        else:
            v['Number of deaths']=int(values[i].text.replace(' ',''))
        print(v)
        if df.empty:
            df=pd.DataFrame.from_dict(v,orient='index').T
        else:
            df=pd.concat([df, pd.DataFrame.from_dict(v,orient='index').T])

{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Albania', 'Number of deaths': 7}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Antigua and Barbuda', 'Number of deaths': 0}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Argentina', 'Number of deaths': 335}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Armenia', 'Number of deaths': 43}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic dise

{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Greece', 'Number of deaths': 14}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Grenada', 'Number of deaths': nan}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guadeloupe', 'Number of deaths': 0}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Guatemala', 'Number of deaths': 1119}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 

{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Saint Vincent and the Grenadines', 'Number of deaths': 2}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Serbia', 'Number of deaths': 17}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Seychelles', 'Number of deaths': nan}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and parasitic diseases', 'Country': 'Singapore', 'Number of deaths': 3}
{'Year': '2000', 'Sex': 'Male', 'Cause Type': 'Communicable, maternal, perinatal and nutritional conditions', 'Cause Death': 'Infectious and

In [27]:
df 

,index,Year,Sex,Cause Type,Cause Death,Country,Number of deaths
0,0.0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Albania,NaN
1,0.0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Antigua and Barbuda,NaN
2,0.0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Argentina,NaN
3,0.0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Armenia,NaN
4,0.0,2020,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Australia,1
...,...,...,...,...,...,...,...
0,NaN,2000,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,United Kingdom,75
0,NaN,2000,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,United States of America,328
0,NaN,2000,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Uruguay,20
0,NaN,2000,Male,"Communicable, maternal, perinatal and nutritio...",Infectious and parasitic diseases,Uzbekistan,359


# 2373 cho 1 nhóm tuổi của 1 nguyên nhân 1 giới tính :) 2373x8x32x3=1,822,464
Số vòng for cần chạy:

- For nguyên nhân

- For nhóm tuổi

- for giới tính

    CODE ở trên và truyền tham số lại